In [1]:
#imports
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from nba_api.stats.endpoints import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from time import *
from datetime import *
from nba_api.stats.library.http import NBAStatsHTTP
import joblib
import scipy.stats as stats
from typing import Optional

# Set the timeout
NBAStatsHTTP.timeout = 120  # Set to 30 seconds; adjust as needed

In [2]:
def player_name(player_ID):
    players = playerindex.PlayerIndex()
    players_data = players.get_data_frames()[0]
    last_name = players_data.loc[players_data['PERSON_ID'] == player_ID, 'PLAYER_LAST_NAME'].iloc[0]
    first_name = players_data.loc[players_data['PERSON_ID'] == player_ID, 'PLAYER_FIRST_NAME'].iloc[0]
    player_Name = first_name + " " + last_name
    return player_Name
# print(player_name(1629008))
def player_id(player_Name):
    players = playerindex.PlayerIndex()
    name = player_Name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players_data = players.get_data_frames()[0]
    player_ID = players_data.loc[(players_data['PLAYER_FIRST_NAME'] == first_name) & (players_data['PLAYER_LAST_NAME'] == last_name), 'PERSON_ID'].iloc[0]
    return player_ID
# Example, should return 1629008
# player_id('Michael Porter Jr.')

In [3]:
def team_id(team_abbrev):
    teams = teamgamelogs.TeamGameLogs()
    teams_data = teams.get_data_frames()[0]
    team_id = teams_data.loc[teams_data['TEAM_ABBREVIATION'] == team_abbrev, 'TEAM_ID'].iloc[0]
    return team_id
# Example usage, should return: 1610612747
# print(team_id('LAL'))
def team_abbrev(team_ID):
    teams = teamdetails.TeamDetails(team_id= team_ID)
    teams_data = teams.get_data_frames()[0]
    team_abbrev = teams_data.loc[teams_data['TEAM_ID'] == team_ID, 'ABBREVIATION'].iloc[0]
    return team_abbrev
# team_abbrev(1610612747)

In [4]:
#function to get player gamelog based on Player Name rather than ID

def player_gamelog(player_name):
    id = player_id(player_name)
    gamelog = playergamelog.PlayerGameLog(player_id= id)
    exc = gamelog.get_data_frames()[0]
    return gamelog.get_data_frames()[0]
# player_gamelog('Michael Porter Jr.')


In [5]:
def playerVplayer(player_name, vs_player_name):
    id = player_id(player_name)
    vid = player_id(vs_player_name)
    gamelog = playervsplayer.PlayerVsPlayer(vs_player_id=vid, player_id=id)
    return gamelog.get_data_frames()

# playerVplayer('Nikola Jokic', 'Rudy Gobert')

In [6]:
def player_team(player_name):
    name = player_name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players = playerindex.PlayerIndex()
    df = players.get_data_frames()[0]
    # df.to_excel('Player_Index.xlsx')
    team_ID = df.loc[(df['PLAYER_LAST_NAME'] == last_name) & (df['PLAYER_FIRST_NAME'] == first_name), 'TEAM_ID'].iloc[0]
    team = team_abbrev(team_ID)
    return team
# player_team('Collin Sexton')

In [ ]:
from datetime import *
#players to add: "CJ McCollum", "Zion Williamson", "De'Andre Hunter", "Andrew Wiggins", "Johnathan Kuminga", "Keyonte George", "Bennedict Mathurin", "Jalen Suggs", "Malik Beasley", "Gradey Dick", "OG Anunoby", "Jimmy Butler", "Derrick White", "Bam Adebayo", "Myles Turner", "Fred VanVleet", "Jerami Grant", "Ivica Zubac", "Dennis Schröder", "Jakob Poeltl", "Payton Pritchard", "Deni Avdija", "Christian Braun", "Josh Hart", "Jarrett Allen", "Deandre Ayton", "P.J. Washington", "Kelly Oubre Jr.", "Naz Reid", "Klay Thompson", "Dyson Daniels", "Bobby Portis", "Russell Westbrook", "Tobias Harris", "Amen Thompson", "D'Angelo Russell", "Dillon Brooks", "Santi Aldama", "Ayo Dosunmu", "Bilal Coulibaly", "Keldon Johnson", "Brook Lopez", "Daniel Gafford", "Rui Hachimura", "Buddy Hield", "Scoot Henderson", "Jabari Smith Jr.", "Keegan Murray", "Ty Jerome"

playerList = [
        "Alperen Sengun", "Anfernee Simons", "Anthony Davis", "Anthony Edwards", "Austin Reaves", "Brandon Miller", "Bradley Beal", "Cade Cunningham", "Cameron Johnson", "Coby White", "Collin Sexton", "Damian Lillard", "Darius Garland", "De'Aaron Fox", "DeMar DeRozan", "Desmond Bane", "Devin Booker", "Domantas Sabonis", "Donovan Mitchell", "Evan Mobley", "Giannis Antetokounmpo", "Jalen Brunson", "Jalen Green", "Jalen Johnson", "Jalen Williams", "Jamal Murray", "James Harden", "Jaren Jackson Jr.", "Jaylen Brown", "Jayson Tatum", "John Collins", "Jordan Poole", "Julius Randle", "Karl-Anthony Towns", "Kevin Durant", "Kristaps Porziņģis", "Kyrie Irving", "LaMelo Ball", "Lauri Markkanen", "LeBron James", "Luka Dončić", "Malik Monk", "Michael Porter Jr.", "Mikal Bridges", "Miles Bridges", "Nikola Jokić", "Nikola Vučević", "Norman Powell", "Pascal Siakam", "Paul George", "RJ Barrett", "Shaedon Sharpe", "Shai Gilgeous-Alexander", "Stephen Curry", "Trae Young", "Tyler Herro", "Tyrese Haliburton", "Tyrese Maxey", "Victor Wembanyama", "Zach LaVine"
    ]
def createPlayerTeamDict():
    playerTeamDict = {}
    for player in playerList:
        team = player_team(player)  # Get the team for this player
        playerTeamDict[player] = team
    return playerTeamDict
# createPlayerTeamDict()

playerTeamDict = {'Alperen Sengun': 'HOU', 'Anfernee Simons': 'POR', 'Anthony Davis': 'LAL', 'Anthony Edwards': 'MIN', 'Austin Reaves': 'LAL', 'Brandon Miller': 'CHA', 'Bradley Beal': 'PHX', 'Cade Cunningham': 'DET', 'Cameron Johnson': 'BKN', 'Coby White': 'CHI', 'Collin Sexton': 'UTA', 'Damian Lillard': 'MIL', 'Darius Garland': 'CLE', "De'Aaron Fox": 'SAC', 'DeMar DeRozan': 'SAC', 'Desmond Bane': 'MEM', 'Devin Booker': 'PHX', 'Domantas Sabonis': 'SAC', 'Donovan Mitchell': 'CLE', 'Evan Mobley': 'CLE', 'Giannis Antetokounmpo': 'MIL', 'Jalen Brunson': 'NYK', 'Jalen Green': 'HOU', 'Jalen Johnson': 'ATL', 'Jalen Williams': 'OKC', 'Jamal Murray': 'DEN', 'James Harden': 'LAC', 'Jaren Jackson Jr.': 'MEM', 'Jaylen Brown': 'BOS', 'Jayson Tatum': 'BOS', 'John Collins': 'UTA', 'Jordan Poole': 'WAS', 'Julius Randle': 'MIN', 'Karl-Anthony Towns': 'NYK', 'Kevin Durant': 'PHX', 'Kristaps Porziņģis': 'BOS', 'Kyrie Irving': 'DAL', 'LaMelo Ball': 'CHA', 'Lauri Markkanen': 'UTA', 'LeBron James': 'LAL', 'Luka Dončić': 'DAL', 'Malik Monk': 'SAC', 'Michael Porter Jr.': 'DEN', 'Mikal Bridges': 'NYK', 'Miles Bridges': 'CHA', 'Nikola Jokić': 'DEN', 'Nikola Vučević': 'CHI', 'Norman Powell': 'LAC', 'Pascal Siakam': 'IND', 'Paul George': 'PHI', 'RJ Barrett': 'TOR', 'Shaedon Sharpe': 'POR', 'Shai Gilgeous-Alexander': 'OKC', 'Stephen Curry': 'GSW', 'Trae Young': 'ATL', 'Tyler Herro': 'MIA', 'Tyrese Haliburton': 'IND', 'Tyrese Maxey': 'PHI', 'Victor Wembanyama': 'SAS', 'Zach LaVine': 'CHI'}
def playedYesterday():
    tlog = leaguegamelog.LeagueGameLog()
    df = tlog.get_data_frames()[0]
    yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    yesterdayGames = df[df['GAME_DATE'] == yesterday]
    yesterdayTeams = yesterdayGames['TEAM_ABBREVIATION'].tolist()
    
    played = []
    for player in playerList:
        team = playerTeamDict.get(player)  # Get the team of the player
        if team in yesterdayTeams:
            played.append(player)
    
    return played
# playedYesterday()
def getPlayerTeam(player_id):
    return playerTeamDict.get(player_name(player_id))
# getPlayerTeam(player_id('Paul George'))

# teams = ['SAS', 'IND', 'POR', 'OKC', 'PHI', 'GSW']
# notPlaying = []
# for player in playerList:
#     team = playerTeamDict.get(player)  # Get the team of the player
#     if team in teams:
#         notPlaying.append(player)
# print(notPlaying)


In [8]:
def getWins(team_id):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    return df['W'][0]
# print(getWins(team_id('BKN')))

def getWinsUpTo(team_id, game_date: Optional[str] = None):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]
    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    if game_date:
        df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] < game_date]
        if df_teamlog.empty:
            return 0
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['W']
# print(getWinsUpTo(team_id('WAS'), 'OCT 24, 2024'))
# print(getWinsUpTo(team_id('WAS')))
def getLosses(team_id):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    return df['L'][0]
# print(getLosses(team_id('WAS')))

def getLossesUpTo(team_id, game_date: Optional[str] = None):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]
    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    if game_date:
        df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] < game_date]
        if df_teamlog.empty:
            return 0
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['L']
# print(getLossesUpTo(team_id('OKC'), 'OCT 25, 2024'))
# print(getLossesUpTo(team_id('OKC')))

def getL5Wins(team_id, game_date: Optional[str] = None):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]
    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    if game_date:
        df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] < game_date]
        if df_teamlog.empty:
            return 0
    df_teamlog = df_teamlog.head(5)
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['W']
# print(getL5Wins(team_id('OKC')))


def getL5PPG(team_id, game_date: Optional[str] = None):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]
    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    if game_date:
        df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] < game_date]
        if df_teamlog.empty:
            return 0
    df_teamlog = df_teamlog.head(5)
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['AVG_PTS']
# getL5PPGRev(team_id=team_id('OKC'), game_date= 'OCT 24, 2024')

In [9]:
def age(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_age = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'AGE'].iloc[0]
    return player_age
# age(player_id('Luka Dončić'))

In [10]:
def height(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_height = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_HEIGHT_INCHES'].iloc[0]
    return player_height
# Example, should return 83(in)
# height(player_id('Nikola Jokić'))

In [11]:
def weight(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_weight = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_WEIGHT'].iloc[0]
    return player_weight
# weight(player_id('Luka Dončić'))

In [12]:
def encode_positions(position):
    roles = {'Primary_Guard': 0, 'Primary_Forward': 0, 'Primary_Center': 0, 'Is_Guard': 0, 'Is_Forward': 0, 'Is_Center': 0}
    primary = position.split('-')[0]
    if 'G' == primary:
        roles['Primary_Guard'] = 1
    elif 'F' == primary:
        roles['Primary_Forward'] = 1
    elif 'C' == primary:
        roles['Primary_Center'] = 1
    if 'G' in position:
        roles['Is_Guard'] = 1
    if 'F' in position:
        roles['Is_Forward'] = 1
    if 'C' in position:
        roles['Is_Center'] = 1
    return pd.Series(roles)
# print(encode_positions('C-G'))
def position(player_ID):
    player_Name = player_name(player_ID)
    name = player_Name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players = playerindex.PlayerIndex()
    df = players.get_data_frames()[0]
    position = df.loc[(df['PLAYER_LAST_NAME'] == last_name) & (df['PLAYER_FIRST_NAME'] == first_name), 'POSITION'].iloc[0]
    return position 
# position(1629008) 
# "LeBron, F"

In [13]:
# Will add a Home_Away column to the gamelog so that the ML model can use that as a variable
def homeOrAway(player_gamelog):
    player_gamelog['Home_Away'] = player_gamelog['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    return player_gamelog
# kingStats = player_gamelog('LeBron James')
# homeOrAway

In [14]:
def extract_team(matchup):
     # The opponent's team abbreviation is after "vs. " or "@ "
    if " vs. " in matchup:
        # Split the string at " vs. " and take the second part
        return matchup.split(" vs. ")[0]
    elif " @ " in matchup:
        # Split the string at " @ " and take the second part
        return matchup.split(" @ ")[0]
    else:
        return "Invalid Matchup Format"
# print(extract_team('LAL @ NYK'))

def extract_opponent(matchup):
    # The opponent's team abbreviation is after "vs. " or "@ "
    if " vs. " in matchup:
        # Split the string at " vs. " and take the second part
        return matchup.split(" vs. ")[1]
    elif " @ " in matchup:
        # Split the string at " @ " and take the second part
        return matchup.split(" @ ")[1]
    else:
        return "Invalid Matchup Format"
# extract_opponent('LAL @ NYK')

In [15]:
def againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev, game_date):
    df_copy = player_gamelog.copy()
    df_copy['GAME_DATE'] = pd.to_datetime(df_copy['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    awaySearchString = home_team_abbrev + ' vs. ' + opp_team_abbrev
    homeSearchString = home_team_abbrev + ' @ ' + opp_team_abbrev
    df_copy = df_copy.iloc[::-1].reset_index(drop=True)
    filtered_log = df_copy[(df_copy['GAME_DATE'] < game_date)]
    newGameLog = filtered_log.loc[filtered_log.MATCHUP.isin([homeSearchString, awaySearchString])]
    newGameLog = newGameLog.sort_values(by='GAME_DATE', ascending=True)
    newGameLog['GAME_DATE'] = newGameLog['GAME_DATE'].apply(lambda x: x.strftime('%b').upper() + x.strftime(' %d, %Y'))
    return newGameLog

# Test
# kingStats = player_gamelog('LeBron James')
# print(againstThisTeam(kingStats, 'LAL', 'SAS', 'JAN 14, 2025'))

def boxStatsAgainstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev):
    new_log = againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev)
    new_log = new_log.loc[:, ['GAME_DATE', 'MATCHUP', 'REB', 'AST', 'STL', 'BLK', 'PTS', 'MIN']]
    return new_log
# test
# dfoxStats = player_gamelog("De'Aaron Fox")
# boxStatsAgainstThisTeam(dfoxStats, 'SAC', 'MEM')

def againstThisTeamLog(player_id, opp_id, game_date):

    cume = cumestatsplayergames.CumeStatsPlayerGames(player_id, vs_team_id_nullable= opp_id)
    df = cume.get_data_frames()[0]
    
    df['GAME_DATE'] = pd.to_datetime(df['MATCHUP'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0])
    specific_date = pd.to_datetime(game_date)
    filtered_df = df[df['GAME_DATE'] < specific_date]
    if filtered_df.empty:
        return pd.DataFrame()
    
    game_ids = filtered_df['GAME_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    
    cume = cumestatsplayer.CumeStatsPlayer(player_id, game_ids_string)
    df = cume.get_data_frames()[1]

    stats_to_average = ['FG', 'FGA', 'FG3', 'FG3A', 'OFF_REB', 'FT', 'FTA']
    for stat in stats_to_average:
        avg_column_name = f'AVG_{stat}'
        df[avg_column_name] = df[stat] / df['GP']

    df['AVG_MIN'] = df['AVG_ACTUAL_MINUTES'] + (df['AVG_ACTUAL_SECONDS'] / 60.0)
    df.rename(columns={'AVG_FG': 'AVG_FGM', 'AVG_FG3': 'AVG_FG3M', 'AVG_OFF_REB': 'AVG_OREB', 'AVG_FT': 'AVG_FTM', 'FT_PCT': 'AVG_FT_PCT', 'FG_PCT': 'AVG_FG_PCT', 'FG3_PCT': 'AVG_FG3_PCT'}, inplace=True)
    return df

# againstThisTeamLog(player_id('Lauri Markkanen'), team_id('MEM'), 'OCT 20, 2024')

In [16]:
def fillMatchupStats(player_log):
    stats_to_average = ['PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'OREB', 'STL', 'MIN', 'PLUS_MINUS'] 

    for i in range(player_log.shape[0]):
        team_abbrev = extract_team(player_log["MATCHUP"][i])
        opp_team_abbrev = extract_opponent(player_log["MATCHUP"][i])
        game_date = player_log['GAME_DATE'].iloc[i]
        matchup_stats = againstThisTeam(player_log, team_abbrev, opp_team_abbrev, game_date)
        if matchup_stats.empty:
            for stat in stats_to_average:
                player_log[f'VS_AVG_{stat}'][i] = player_log[f'AVG_{stat}'][i]
        else:
            for stat in stats_to_average:
                player_log.loc[i, f'VS_AVG_{stat}'] = matchup_stats[stat].mean()
    return player_log

# df = pd.read_csv('v2.2 csvs/Alperen Sengun.csv')
# fillMatchupStats(df)

In [17]:
def getOFFefficiency(team_ID):
    offenses = teamestimatedmetrics.TeamEstimatedMetrics()
    df = offenses.get_data_frames()[0]
    offEfficiency = df.loc[df['TEAM_ID'] == team_ID, 'E_OFF_RATING'].iloc[0]
    return offEfficiency
# print(getOFFefficiency(team_id('CLE')))
def getPPG(team_ID):
    team_stats = leaguestandingsv3.LeagueStandingsV3()
    df_teamstats = team_stats.get_data_frames()[0]
    team_row = df_teamstats[df_teamstats['TeamID'] == team_ID]
    if not team_row.empty:
        totalPTSA = team_row.iloc[0]['PointsPG']
        return totalPTSA
    else:
        print("No data found for the specified city")
        return None
# print(getPPG(team_id('LAL')))
def getPPGUpTo(team_id, date):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]

    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    date = pd.to_datetime(date)
    df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] < date]
    if df_teamlog.empty:
        return 0
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)

    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['AVG_PTS']
# print(getPPGUpTo(team_id('HOU'), 'SEP 21, 2024'))

In [18]:
# Note: the numbers in the def efficiency do not match the numbers on the official website, yet they are in the correct rankings, (i.e OKC's def efficiency is #1 in this data, and #1 on the official data, however they have differing def efficiency numbers, one could be 102.9 the other 103.1)
def getDEFefficiency(team_ID):
    defenses = teamestimatedmetrics.TeamEstimatedMetrics()
    df = defenses.get_data_frames()[0]
    defEfficiency = df.loc[df['TEAM_ID'] == team_ID, 'E_DEF_RATING'].iloc[0]
    return defEfficiency
# print(getDEFefficiency(team_id('LAL')))
    
def getPPG_allowed(team_ID):
    team_stats = leaguestandingsv3.LeagueStandingsV3()
    df_teamstats = team_stats.get_data_frames()[0]
    team_row = df_teamstats[df_teamstats['TeamID'] == team_ID]
    if not team_row.empty:
        totalPTSA = team_row.iloc[0]['OppPointsPG']
        return totalPTSA
    else:
        print("No data found for the specified city")
        return None
# Example Usage, Output: 113.70588235294117
# print(getPPG_allowed(team_id('OKC')))

def getPPG_allowedUpTo(team_id, date):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]

    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    date = pd.to_datetime(date)
    df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] < date]
    if df_teamlog.empty:
        return 0
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)

    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[1]['AVG_PTS']
# getPPG_allowedUpTo(team_id('OKC'), 'OCT 10, 2024')


In [19]:
def last5ID(player_id, date):
    date = pd.to_datetime(date)

    df = player_gamelog(player_name(player_id))
    
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    df = df[df['GAME_DATE'] < date]
    df = df['Game_ID']
    df = df.head(5)

    teamlog = teamgamelog.TeamGameLog(team_id= team_id(getPlayerTeam(player_id)))
    df_team = teamlog.get_data_frames()[0]
    df_team['GAME_DATE'] = pd.to_datetime(df_team['GAME_DATE'])
    df_team = df_team[df_team['GAME_DATE'] < date]
    df_team = df_team['Game_ID']
    df_team = df_team.head(5)
    
    common_games = pd.merge(df, df_team, on='Game_ID', how='inner')
    return common_games['Game_ID'].tolist()
# print(last5ID(player_id('Shai Gilgeous-Alexander'),  date='JAN 19, 2025'))
def last5Stats(player_id, date):
    game_ids = last5ID(player_id, date)
    game_ids_string = '|'.join(game_ids)
    if game_ids_string == '':
        return pd.DataFrame()
    cume = cumestatsplayer.CumeStatsPlayer(player_id, game_ids_string)
    df = cume.get_data_frames()[1]
    stats_to_average = ['FG', 'FGA', 'FG3', 'FG3A', 'OFF_REB', 'FT', 'FTA']
    for stat in stats_to_average:
        avg_column_name = f'AVG_{stat}'
        df[avg_column_name] = df[stat] / df['GP']

    df['AVG_MIN'] = df['AVG_ACTUAL_MINUTES'] + (df['AVG_ACTUAL_SECONDS'] / 60.0)
    df.rename(columns={'AVG_FG': 'AVG_FGM', 'AVG_FG3': 'AVG_FG3M', 'AVG_OFF_REB': 'AVG_OREB', 'AVG_FT': 'AVG_FTM', 'FT_PCT': 'AVG_FT_PCT', 'FG_PCT': 'AVG_FG_PCT', 'FG3_PCT': 'AVG_FG3_PCT'}, inplace=True)
    return df
# print(last5Stats(player_id('Lauri Markkanen'), date='JAN 28, 2025'))
def allPlayerGameIDS(player_id, game_date: Optional[str] = None):
    cume = cumestatsplayergames.CumeStatsPlayerGames(player_id)
    df = cume.get_data_frames()[0]

    teamlog = teamgamelog.TeamGameLog(team_id= team_id(getPlayerTeam(player_id)))
    df_team = teamlog.get_data_frames()[0]
    if game_date:
        df_team['GAME_DATE'] = pd.to_datetime(df_team['GAME_DATE'])
        game_date = pd.to_datetime(game_date)
        df_team = df_team[df_team['GAME_DATE'] < game_date]
        
    df_team.rename(columns={'Game_ID': 'GAME_ID'}, inplace=True)
    df_team = df_team['GAME_ID']
    df = df['GAME_ID']

    common_games = pd.merge(df, df_team, on='GAME_ID', how='inner')
    return common_games['GAME_ID'].tolist()
# allPlayerGameIDS(player_id('Donovan Mitchell'), 'OCT 25, 2024')
def seasonAverages(player_id, game_date: Optional[str] = None):
    # Can be up to a certain date or just overall averages
    game_ids = allPlayerGameIDS(player_id, game_date)
    game_ids_string = '|'.join(game_ids)
    if game_ids_string == '':
        return pd.DataFrame()
    cume = cumestatsplayer.CumeStatsPlayer(player_id, game_ids_string)
    df = cume.get_data_frames()[1]
    stats_to_average = ['FG', 'FGA', 'FG3', 'FG3A', 'OFF_REB', 'FT', 'FTA']
    for stat in stats_to_average:
        avg_column_name = f'AVG_{stat}'
        df[avg_column_name] = df[stat] / df['GP']

    df['AVG_MIN'] = df['AVG_ACTUAL_MINUTES'] + (df['AVG_ACTUAL_SECONDS'] / 60.0)
    df.rename(columns={'AVG_FG': 'AVG_FGM', 'AVG_FG3': 'AVG_FG3M', 'AVG_OFF_REB': 'AVG_OREB', 'AVG_FT': 'AVG_FTM', 'FT_PCT': 'AVG_FT_PCT', 'FG_PCT': 'AVG_FG_PCT', 'FG3_PCT': 'AVG_FG3_PCT'}, inplace=True)
    return df
# seasonAverages(player_id('Shai Gilgeous-Alexander'), 'SEP 01, 2024')


In [20]:
def concatAllCSVS(folder):
    # List to hold dataframes
    dfs = []
    
    # Iterate over each file in the directory
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            # Construct full file path
            file_path = os.path.join(folder, filename)
            # Read the CSV file and append to list
            df = pd.read_csv(file_path, index_col=0)
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            df = df.iloc[7:].reset_index(drop=True)
            average_points = df['PTS'].mean()
            df['target'] = (df['PTS'] > average_points).astype(int)
            df.dropna(inplace=True)
            dfs.append(df)
    
    # Concatenate all dataframes in the list
    full_df = pd.concat(dfs, ignore_index=True)
    return full_df
# master_csv = concatAllCSVS('players csvs')

In [21]:
# NOTE!!!!!!!!!!!! THIS SHOULD ONLY BE USED ONCE TO CREATE A NEW CSV FOR A PLAYER. ONCE A CSV HAS BEEN CREATED, USE THE updateStats METHOD TO UPDATE THEIR MOST CURRENT GAME.

def importantStatsAccurate(player_name):
    player_gameLog = player_gamelog(player_name)
    player_gameLog = player_gameLog.iloc[::-1].reset_index(drop=True)
    playerID = player_id(player_name)
    player_gameLog['TEAM_ABBREVIATION'] = player_gameLog['MATCHUP'].apply(extract_team)
    player_gameLog['TEAM_ID'] = player_gameLog['TEAM_ABBREVIATION'].apply(team_id)
    player_gameLog['OPP_TEAM'] = player_gameLog['MATCHUP'].apply(extract_opponent)
    player_gameLog['OPP_TEAMID'] = player_gameLog['OPP_TEAM'].apply(team_id)
    homeOrAway(player_gameLog)
    player_gameLog['POSITION'] = player_gameLog['Player_ID'].apply(position)
    encoded_positions = player_gameLog['POSITION'].apply(encode_positions)
    player_gameLog = pd.concat([player_gameLog, encoded_positions], axis=1)
    player_gameLog['AGE'] = player_gameLog['Player_ID'].apply(age)
    player_gameLog['HEIGHT'] = player_gameLog['Player_ID'].apply(height)
    player_gameLog['WEIGHT'] = player_gameLog['Player_ID'].apply(weight)
    player_gameLog['TEAM_W'] = player_gameLog.apply(lambda row: getWinsUpTo(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['TEAM_L'] = player_gameLog.apply(lambda row: getLossesUpTo(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['PPG'] = player_gameLog.apply(lambda row: getPPGUpTo(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['L5_W'] = player_gameLog.apply(lambda row: getL5Wins(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['L5_L'] = 5 - player_gameLog['L5_W']
    player_gameLog['L5_PPG'] = player_gameLog.apply(lambda row: getL5PPG(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_TEAM_W'] = player_gameLog.apply(lambda row: getWinsUpTo(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_TEAM_L'] = player_gameLog.apply(lambda row: getLossesUpTo(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_PPG_ALLOWED'] = player_gameLog.apply(lambda row: getPPG_allowedUpTo(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_L5_W'] = player_gameLog.apply(lambda row: getL5Wins(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_L5_L'] = 5 - player_gameLog['OPP_L5_W']
    player_gameLog['OPP_L5_PPG'] = player_gameLog.apply(lambda row: getL5PPG(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)

    window_size = 5
    stats_to_average = ['PLUS_MINUS']
    for stat in stats_to_average:
        player_gameLog[f'MA_{stat}'] = player_gameLog[stat].rolling(window=window_size, min_periods=1).mean().shift(0)
        player_gameLog[f'AVG_{stat}'] = player_gameLog[stat].expanding().mean().shift(1)
        
    for stat in stats_to_average:
        player_gameLog.at[0, f'AVG_{stat}'] = player_gameLog.at[0, stat]

    stats_to_average = ['PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'STL', 'MIN']
    for stat in stats_to_average:
        player_gameLog[f'MA_{stat}'] = np.nan 
        player_gameLog[f'AVG_{stat}'] = np.nan
        player_gameLog[f'VS_{stat}'] = np.nan
    for index, row in player_gameLog.iterrows():
        game_date = row['GAME_DATE']
        opp_id = row['OPP_TEAMID']
        if index == 0:
            for stat in stats_to_average:
                player_gameLog[f'AVG_{stat}'] = player_gameLog[f'{stat}'].iloc[0] 
            continue
        season_stats_df = seasonAverages(playerID, game_date)
        l5_stats_df = last5Stats(playerID, game_date) 
        vs_stats_df = againstThisTeamLog(playerID, opp_id, game_date)
        if l5_stats_df.empty:
            if vs_stats_df.empty:
                for stat in stats_to_average:
                    player_gameLog.at[index, f'AVG_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
                    player_gameLog.at[index, f'MA_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
                    player_gameLog.at[index, f'VS_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
            else:
                for stat in stats_to_average:
                    player_gameLog.at[index, f'AVG_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
                    player_gameLog.at[index, f'MA_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
                    player_gameLog.at[index, f'VS_{stat}'] = vs_stats_df[f'AVG_{stat}'].iloc[0]
        else:
            if vs_stats_df.empty:
                for stat in stats_to_average:
                    player_gameLog.at[index, f'AVG_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
                    player_gameLog.at[index, f'MA_{stat}'] = l5_stats_df[f'AVG_{stat}'].iloc[0]
                    player_gameLog.at[index, f'VS_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
            else:
                for stat in stats_to_average:
                    player_gameLog.at[index, f'AVG_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
                    player_gameLog.at[index, f'MA_{stat}'] = l5_stats_df[f'AVG_{stat}'].iloc[0]
                    player_gameLog.at[index, f'VS_{stat}'] = vs_stats_df[f'AVG_{stat}'].iloc[0]
    # fillMatchupStats(player_gameLog)
    player_gameLog['target'] = (player_gameLog['PTS'] > player_gameLog['AVG_PTS']).astype(int)

    player_gameLog.reset_index(drop=True, inplace=True)
    
    return player_gameLog

# playedList = playedYesterday()
# print(playedList)
#still to add: 



def save_last_updated_player(player_name):
    try:
        with open("last_updated_player.txt", "w") as file:
            file.write(player_name)
    except Exception as e:
        print(f"Error saving last updated player: {e}")

def get_last_updated_player():
    try:
        with open("last_updated_player.txt", "r") as file:
            return file.read().strip()
    except Exception as e:
        print(f"Could not read last updated player file: {e}")
        return None

players_list_all = [
   "Damian Lillard", "Giannis Antetokounmpo",  "John Collins", "Lauri Markkanen", "Shaedon Sharpe", "Stephen Curry"
] 
# last_updated_player = get_last_updated_player()
# start_index = players_list_all.index(last_updated_player) + 1 if last_updated_player else 0

# max_retries = 50  # Maximum number of retries per player

# for player in players_list_all[start_index:]:
#     attempts = 0
#     while attempts < max_retries:
#         try:
#             file_path = os.path.join('v2.4 csvs', player + '.csv')
#             df_updated = importantStatsAccurate(player)
#             df_updated.to_csv(file_path)
#             save_last_updated_player(player)
#             print("This player's csv was created: ", player)
#             break  # Break out of the retry loop on success
#         except Exception as e:
#             attempts += 1
#             print(f"Error updating {player}, attempt {attempts} of {max_retries}: {e}")
#             if attempts == max_retries:
#                 print(f"Max retries reached for {player}. Moving to the next player.")
#                 break

# Optional: Concatenate all CSVs into one after updating
# masterTESTcsv = concatAllCSVS('v2.4 csvs')
# masterTESTcsv.to_csv('masterCSV_2025-01-29 v2.4.csv')

In [22]:
# THIS IS TO CREATE A MODEL WITH EVERYONE'S CSVS CONCATTED
def createModelALL():
    file_path = 'masterCSV_2025-01-29 v2.4.csv'  # Specify your CSV file name here
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)

    # Features including PLAYER_ID
    categorical_features = ['MATCHUP', 'Player_ID', 'TEAM_ID', 'OPP_TEAMID']
    classifier = Pipeline([
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
            remainder='passthrough')),
        ('classifier', LogisticRegression(solver = 'saga', max_iter=50000, warm_start=True))
    ])
    # 'TEAM_ID', 'OPP_TEAMID',
    X = df[["Player_ID", "MATCHUP", "TEAM_ID","OPP_TEAMID","Home_Away","Primary_Guard","Primary_Forward","Primary_Center","Is_Guard","Is_Forward","Is_Center","AGE","HEIGHT","WEIGHT","TEAM_W","TEAM_L","PPG","L5_W","L5_L","L5_PPG","OPP_TEAM_W","OPP_TEAM_L","OPP_PPG_ALLOWED","OPP_L5_W","OPP_L5_L","OPP_L5_PPG","MA_PLUS_MINUS","AVG_PLUS_MINUS","MA_PTS","AVG_PTS","VS_PTS","MA_FGM","AVG_FGM","VS_FGM","MA_FGA","AVG_FGA","VS_FGA","MA_FG_PCT","AVG_FG_PCT","VS_FG_PCT","MA_FG3M","AVG_FG3M","VS_FG3M","MA_FG3A","AVG_FG3A","VS_FG3A","MA_FG3_PCT","AVG_FG3_PCT","VS_FG3_PCT","MA_FTM","AVG_FTM","VS_FTM","MA_FTA","AVG_FTA","VS_FTA","MA_FT_PCT","AVG_FT_PCT","VS_FT_PCT","MA_OREB","AVG_OREB","VS_OREB","MA_STL","AVG_STL","VS_STL","MA_MIN","AVG_MIN","VS_MIN"]]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    # Save the model
    joblib.dump(classifier, 'master_model_lib v2.4_saga.pkl')
# createModelALL()

In [23]:
def testNewAccuracy(iterations=500):
    # Load the dataset
    df = pd.read_csv('masterCSV_2025-01-29 v2.4.csv')

    # Define features and target, 
    X = df[["Player_ID", "MATCHUP", "TEAM_ID","OPP_TEAMID","Home_Away","Primary_Guard","Primary_Forward","Primary_Center","Is_Guard","Is_Forward","Is_Center","AGE","HEIGHT","WEIGHT","TEAM_W","TEAM_L","PPG","L5_W","L5_L","L5_PPG","OPP_TEAM_W","OPP_TEAM_L","OPP_PPG_ALLOWED","OPP_L5_W","OPP_L5_L","OPP_L5_PPG","MA_PLUS_MINUS","AVG_PLUS_MINUS","MA_PTS","AVG_PTS","VS_PTS","MA_FGM","AVG_FGM","VS_FGM","MA_FGA","AVG_FGA","VS_FGA","MA_FG_PCT","AVG_FG_PCT","VS_FG_PCT","MA_FG3M","AVG_FG3M","VS_FG3M","MA_FG3A","AVG_FG3A","VS_FG3A","MA_FG3_PCT","AVG_FG3_PCT","VS_FG3_PCT","MA_FTM","AVG_FTM","VS_FTM","MA_FTA","AVG_FTA","VS_FTA","MA_FT_PCT","AVG_FT_PCT","VS_FT_PCT","MA_OREB","AVG_OREB","VS_OREB","MA_STL","AVG_STL","VS_STL","MA_MIN","AVG_MIN","VS_MIN"]]
    y = df['target']

    # Load the trained model
    model = joblib.load('master_model_lib v2.4.pkl')

    # List to store accuracies
    accuracies = []

    # Run the testing process multiple times
    for i in range(iterations):
        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=i)  # Vary random_state for different splits

        # Predict and calculate accuracy
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        accuracies.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracies)
    print(f"\nAverage Accuracy over {iterations} iterations: {average_accuracy:.4f}")

    lowest = min(accuracies)
    highest = max(accuracies)
    print("Lowest accuracy:", lowest)
    print("Highest accuracy:", highest)
    z_5th = stats.norm.ppf(0.05)  # 5th percentile Z-score
    z_95th = stats.norm.ppf(0.95)  # 95th percentile Z-score

    # Convert Z-scores to actual values
    value_5th = np.mean(accuracies) + z_5th * np.std(accuracies)
    value_95th = np.mean(accuracies) + z_95th * np.std(accuracies)

    print(f"5th percentile value: {value_5th}")
    print(f"95th percentile value: {value_95th}")

    print('STD. DEV:', np.std(accuracies))

# Run the function with the desired number of iterations
testNewAccuracy(iterations=500)


Average Accuracy over 500 iterations: 0.7158
Lowest accuracy: 0.4782608695652174
Highest accuracy: 0.7698209718670077
5th percentile value: 0.6778360148549644
95th percentile value: 0.7538468495951635
STD. DEV: 0.023105653139810237


In [24]:
def warm_start_train(old_model_path, data, output_model_path):
    """
    Train LogisticRegression with warm start using combined old and new data.

    Parameters:
    - old_model_path: Path to the previously saved model.
    - new_data_path: Path to the new data CSV.
    - master_csv_path: Path to the master dataset CSV.
    - output_model_path: Path to save the updated model.
    """
    # Load new data
    new_data = pd.read_csv(data)
    new_data = new_data.iloc[:, 1:] 
    new_data = new_data.iloc[7:].reset_index(drop=True)
    # Define features and target
    X = new_data[["Player_ID", "TEAM_ID","OPP_TEAMID","Home_Away","Primary_Guard","Primary_Forward","Primary_Center","Is_Guard","Is_Forward","Is_Center","AGE","HEIGHT","WEIGHT","TEAM_W","TEAM_L","PPG","L5_W","L5_L","L5_PPG","OPP_TEAM_W","OPP_TEAM_L","OPP_PPG_ALLOWED","OPP_L5_W","OPP_L5_L","OPP_L5_PPG","MA_PLUS_MINUS","AVG_PLUS_MINUS","MA_PTS","AVG_PTS","VS_PTS","MA_FGM","AVG_FGM","VS_FGM","MA_FGA","AVG_FGA","VS_FGA","MA_FG_PCT","AVG_FG_PCT","VS_FG_PCT","MA_FG3M","AVG_FG3M","VS_FG3M","MA_FG3A","AVG_FG3A","VS_FG3A","MA_FG3_PCT","AVG_FG3_PCT","VS_FG3_PCT","MA_FTM","AVG_FTM","VS_FTM","MA_FTA","AVG_FTA","VS_FTA","MA_FT_PCT","AVG_FT_PCT","VS_FT_PCT","MA_OREB","AVG_OREB","VS_OREB","MA_STL","AVG_STL","VS_STL","MA_MIN","AVG_MIN","VS_MIN"]]
    y = new_data['target']
    # Train the model on the combined data
    model = joblib.load(old_model_path)
    model.fit(X, y)
    print("Model training completed.")

    # Save the updated model
    joblib.dump(model, output_model_path)
    print(f"Updated model saved to {output_model_path}")

# Example usage
# warm_start_train(
#     old_model_path='master_model_lib v2.3.pkl', 
#     data = 'masterCSV_2025-01-25 v2.3.csv',
#     output_model_path='master_model_lib v2.3 warm.pkl'
# )


In [25]:
import datetime
def getPredictData(player_name, matchup):
    # Construct the file path using the player's name
    file_path = f'v2.4 csvs/{player_name}.csv' 
    df = pd.read_csv(file_path)
    df = df.iloc[:, 1:] 
    window_size = 5
    stats_to_average = ['PLUS_MINUS']
    for stat in stats_to_average:
        df[f'MA_{stat}'] = df[stat].rolling(window=window_size, min_periods=1).mean().shift(0)
        df[f'AVG_{stat}'] = df[stat].expanding().mean().shift(1)
    # print(df)
    df = df.iloc[7:].reset_index(drop=True)
    pred_cols = ["Player_ID", "TEAM_ID","OPP_TEAMID","Home_Away","Primary_Guard","Primary_Forward","Primary_Center","Is_Guard","Is_Forward","Is_Center","AGE","HEIGHT","WEIGHT","TEAM_W","TEAM_L","PPG","L5_W","L5_L","L5_PPG","OPP_TEAM_W","OPP_TEAM_L","OPP_PPG_ALLOWED","OPP_L5_W","OPP_L5_L","OPP_L5_PPG","MA_PLUS_MINUS","AVG_PLUS_MINUS","MA_PTS","AVG_PTS","VS_PTS","MA_FGM","AVG_FGM","VS_FGM","MA_FGA","AVG_FGA","VS_FGA","MA_FG_PCT","AVG_FG_PCT","VS_FG_PCT","MA_FG3M","AVG_FG3M","VS_FG3M","MA_FG3A","AVG_FG3A","VS_FG3A","MA_FG3_PCT","AVG_FG3_PCT","VS_FG3_PCT","MA_FTM","AVG_FTM","VS_FTM","MA_FTA","AVG_FTA","VS_FTA","MA_FT_PCT","AVG_FT_PCT","VS_FT_PCT","MA_OREB","AVG_OREB","VS_OREB","MA_STL","AVG_STL","VS_STL","MA_MIN","AVG_MIN","VS_MIN"]
    pred_df = pd.DataFrame(columns=pred_cols)
    if ' @ ' in matchup:
        teams = matchup.split(' @ ')
        home_away = 0  
    elif ' vs. ' in matchup:
        teams = matchup.split(' vs. ')
        home_away = 1 
    playerID = df['Player_ID'].iloc[0]
    opponentID = team_id(extract_opponent(matchup))
    pred_df.at[0, 'MATCHUP'] = matchup
    pred_df.at[0, 'OPP_TEAM']= extract_opponent(matchup)
    pred_df.at[0, 'OPP_TEAMID'] = opponentID
    pred_df.at[0, 'OPP_PPG_ALLOWED'] = getPPG_allowed(team_id(teams[1]))
    # pred_df['OPP_DEF_EFFICIENCY'] = getDEFefficiency(team_id(teams[1]))
    pred_df.at[0, 'OPP_TEAM_W'] = getWins(team_id(teams[1]))
    pred_df.at[0, 'OPP_TEAM_L'] = getLosses(team_id(teams[1]))
    pred_df.at[0, 'OPP_L5_W'] = getL5Wins(team_id(teams[1]))
    pred_df.at[0, 'OPP_L5_L'] = 5 - pred_df.at[0, 'OPP_L5_W']
    pred_df.at[0, 'OPP_L5_PPG'] = getL5PPG(team_id(teams[1]))
    pred_df.at[0, 'Home_Away'] = home_away
    pred_df.at[0, 'PPG'] = getPPG(team_id(teams[0]))
    pred_df.at[0, 'MA_PLUS_MINUS'] = df['MA_PLUS_MINUS'].iloc[len(df)-1]
    pred_df.at[0, 'AVG_PLUS_MINUS'] = df['AVG_PLUS_MINUS'].iloc[len(df)-1]
    pred_df.at[0, 'Primary_Guard'] = df['Primary_Guard'].iloc[0]
    pred_df.at[0, 'Primary_Forward'] = df['Primary_Forward'].iloc[0]
    pred_df.at[0, 'Primary_Center'] = df['Primary_Center'].iloc[0]
    pred_df.at[0, 'Is_Guard'] = df['Is_Guard'].iloc[0]
    pred_df.at[0, 'Is_Forward'] = df['Is_Forward'].iloc[0]
    pred_df.at[0, 'Is_Center'] = df['Is_Center'].iloc[0]
    pred_df.at[0, 'Player_ID'] = playerID
    pred_df.at[0, 'TEAM_ID'] = team_id(teams[0])
    pred_df.at[0, 'AGE'] = df['AGE'].iloc[0]
    pred_df.at[0, 'HEIGHT'] = df['HEIGHT'].iloc[0]
    pred_df.at[0, 'WEIGHT'] = df['WEIGHT'].iloc[0]
    pred_df.at[0, 'TEAM_W'] = getWins(team_id(teams[0]))
    pred_df.at[0, 'TEAM_L'] = getLosses(team_id(teams[0]))
    pred_df.at[0, 'L5_W'] = getL5Wins(team_id(teams[0]))
    pred_df.at[0, 'L5_L'] = 5 - pred_df.at[0, 'L5_W']
    pred_df.at[0, 'L5_PPG'] = getL5PPG(team_id(teams[0]))
    
    today= datetime.date.today().strftime('%b %d, %Y').upper()
    l5stats = last5Stats(playerID, today)
    seasonStats = seasonAverages(playerID)
    vsStats = againstThisTeamLog(playerID, opponentID, today)
    stats_to_average = ['PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'STL', 'MIN']
    if l5stats.empty:
        if vsStats.empty:
            for stat in stats_to_average:
                pred_df.at[0, f'AVG_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                pred_df.at[0, f'MA_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                pred_df.at[0, f'VS_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
        else:
            for stat in stats_to_average:
                pred_df.at[0, f'AVG_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                pred_df.at[0, f'MA_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                pred_df.at[0, f'VS_{stat}'] = vsStats[f'AVG_{stat}'].iloc[0]
    else:
        if vsStats.empty:
            for stat in stats_to_average:
                pred_df.at[0, f'AVG_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                pred_df.at[0, f'MA_{stat}'] = l5stats[f'AVG_{stat}'].iloc[0]
                pred_df.at[0, f'VS_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
        else:
            for stat in stats_to_average:
                pred_df.at[0, f'AVG_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                pred_df.at[0, f'MA_{stat}'] = l5stats[f'AVG_{stat}'].iloc[0]
                pred_df.at[0, f'VS_{stat}'] = vsStats[f'AVG_{stat}'].iloc[0]

    return pred_df
# getPredictData("Giannis Antetokounmpo", 'MIL @ POR').to_excel('PredictDatav2.4.xlsx')
def predict_performance(new_data):
    classifier = joblib.load('master_model_lib v2.4.pkl')
    
    predictions = classifier.predict(new_data)
    probabilities = classifier.predict_proba(new_data)
    
    return predictions, probabilities

# Example usage
# Assuming 'new_data' is DataFrame with the same features used in training
new_data = getPredictData("Cade Cunningham", 'DET @ IND')
performance_predictions, performance_probabilities = predict_performance(new_data)
print("Line: " + str(new_data['AVG_PTS'].values[0]) + " PTS")
print("Predictions:", performance_predictions)  
print("Probabilities:", performance_probabilities)

Line: 25.0 PTS
Predictions: [0]
Probabilities: [[0.57294163 0.42705837]]


In [26]:
def kelly_criterion(bankroll, vegas_odds, probability):
    if vegas_odds > 0:
        b = vegas_odds / 100
    else:
        b = -100 / vegas_odds
    
    # Calculate the Kelly fraction
    q = 1 - probability
    kelly_fraction = (b * probability - q) / b
    
    # Calculate the bet size
    bet_size = kelly_fraction * bankroll
    
    return max(0, bet_size)  # Ensure the bet size is not negative

bankroll = 45  # $1,000 available for betting
vegas_odds = -130 # +150 odds
probability = 0.87551534# 55% chance of winning

bet_amount = kelly_criterion(bankroll, vegas_odds, probability)
print(f"You should bet ${bet_amount:.2f}")

You should bet $32.12


In [38]:

def update(player_name):
    file_path = os.path.join('v2.4 csvs', player_name + '.csv')
    df = pd.read_csv(file_path)
    log = player_gamelog(player_name)
    log = log.head(1)
    if df['GAME_DATE'].iloc[len(df)-1] == log['GAME_DATE'].iloc[0]:
        return pd.DataFrame()
    cols_to_fill = ["TEAM_ABBREVIATION","TEAM_ID","OPP_TEAM","OPP_TEAMID","Home_Away","POSITION","Primary_Guard","Primary_Forward","Primary_Center","Is_Guard","Is_Forward","Is_Center","AGE","HEIGHT","WEIGHT","TEAM_W","TEAM_L","PPG","L5_W","L5_L","L5_PPG","OPP_TEAM_W","OPP_TEAM_L","OPP_PPG_ALLOWED","OPP_L5_W","OPP_L5_L","OPP_L5_PPG","MA_PLUS_MINUS","AVG_PLUS_MINUS","MA_PTS","AVG_PTS","VS_PTS","MA_FGM","AVG_FGM","VS_FGM","MA_FGA","AVG_FGA","VS_FGA","MA_FG_PCT","AVG_FG_PCT","VS_FG_PCT","MA_FG3M","AVG_FG3M","VS_FG3M","MA_FG3A","AVG_FG3A","VS_FG3A","MA_FG3_PCT","AVG_FG3_PCT","VS_FG3_PCT","MA_FTM","AVG_FTM","VS_FTM","MA_FTA","AVG_FTA","VS_FTA","MA_FT_PCT","AVG_FT_PCT","VS_FT_PCT","MA_OREB","AVG_OREB","VS_OREB","MA_STL","AVG_STL","VS_STL","MA_MIN","AVG_MIN","VS_MIN","target"]
    
    empty_df = pd.DataFrame(columns=cols_to_fill, index=log.index) 
    playerID = log['Player_ID'].iloc[0]
    update_df = pd.concat([log, empty_df], axis=1)
    game_date = log['GAME_DATE'].iloc[0]
    update_df['TEAM_ABBREVIATION'] = log['MATCHUP'].apply(extract_team)
    update_df['TEAM_ID'] = update_df['TEAM_ABBREVIATION'].apply(team_id)
    team_ID = update_df['TEAM_ID'].iloc[0]
    update_df['OPP_TEAM'] = log['MATCHUP'].apply(extract_opponent)
    update_df['OPP_TEAMID'] = update_df['OPP_TEAM'].apply(team_id)
    opp_ID = update_df['OPP_TEAMID'].iloc[0]
    homeOrAway(update_df)
    update_df['POSITION'] = log['Player_ID'].apply(position)
    update_df['Primary_Guard'] = df['Primary_Guard'].iloc[0]
    update_df['Primary_Forward'] = df['Primary_Forward'].iloc[0]
    update_df['Primary_Center'] = df['Primary_Center'].iloc[0]
    update_df['Is_Guard'] = df['Is_Guard'].iloc[0]
    update_df['Is_Forward'] = df['Is_Forward'].iloc[0]
    update_df['Is_Center'] = df['Is_Center'].iloc[0]
    update_df['AGE'] = df['AGE'].iloc[0]
    update_df['HEIGHT'] = df['HEIGHT'].iloc[0]
    update_df['WEIGHT'] = df['WEIGHT'].iloc[0]
    update_df['TEAM_W'] = getWinsUpTo(team_ID, game_date)
    update_df['TEAM_L'] = getLossesUpTo(team_ID, game_date)
    update_df['PPG'] = getPPGUpTo(team_ID, game_date)
    update_df['L5_W'] = getL5Wins(team_ID, game_date)
    update_df['L5_L'] = 5 - update_df['L5_W']
    update_df['L5_PPG'] = getL5PPG(team_ID, game_date)
    update_df['OPP_TEAM_W'] = getWinsUpTo(opp_ID, game_date)
    update_df['OPP_TEAM_L'] = getLossesUpTo(opp_ID, game_date)
    update_df['OPP_PPG_ALLOWED'] = getPPG_allowedUpTo(opp_ID, game_date)
    update_df['OPP_L5_W'] = getL5Wins(opp_ID, game_date)
    update_df['OPP_L5_L'] = 5 - update_df['OPP_L5_W']
    update_df['OPP_L5_PPG'] = getL5PPG(opp_ID, game_date)
    update_df['MA_PLUS_MINUS'] = (df['PLUS_MINUS'].tail(4).sum() + log['PLUS_MINUS'].iloc[0]) / 5
    update_df['AVG_PLUS_MINUS'] = df['PLUS_MINUS'].mean()

    l5stats = last5Stats(playerID, game_date)
    seasonStats = seasonAverages(playerID, game_date)
    vsStats = againstThisTeamLog(playerID, opp_ID, game_date)
    stats_to_average = ['PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'STL', 'MIN']
    if l5stats.empty:
        if vsStats.empty:
            for stat in stats_to_average:
                update_df[f'AVG_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                update_df[f'MA_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                update_df[f'VS_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
        else:
            for stat in stats_to_average:
                update_df[f'AVG_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                update_df[f'MA_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                update_df[f'VS_{stat}'] = vsStats[f'AVG_{stat}'].iloc[0]
    else:
        if vsStats.empty:
            for stat in stats_to_average:
                update_df[f'AVG_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                update_df[f'MA_{stat}'] = l5stats[f'AVG_{stat}'].iloc[0]
                update_df[f'VS_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
        else:
            for stat in stats_to_average:
                update_df[f'AVG_{stat}'] = seasonStats[f'AVG_{stat}'].iloc[0]
                update_df[f'MA_{stat}'] = l5stats[f'AVG_{stat}'].iloc[0]
                update_df[f'VS_{stat}'] = vsStats[f'AVG_{stat}'].iloc[0]
    
    update_df['target'] = (update_df['PTS'] > update_df['AVG_PTS']).astype(int)

    update_df.reset_index(drop=True, inplace=True)
    return update_df
    
played_yesterday = playedYesterday()
for player in played_yesterday:
    file_path = os.path.join('v2.4 csvs', player + '.csv')
    main_df = pd.read_csv(file_path)
    update_df = update(player)
    if update_df.empty:
        print("This player's CSV is already up to date: ", player)
        continue
    result_df = pd.concat([main_df, update_df], ignore_index=True)
    result_df.to_csv(file_path)
    print("This player's csv was updated: ", player)

TypeError: 'list' object is not callable